#Báo cáo máy học
Họ tên sinh viên: Trương Nhật Tiến _
Mssv: 20063731

Áp dụng những thuật toán giới thiệu trong bài giảng, so sánh đánh giá sự hiệu quả của các thuật toán đó và đưa ra những thuật toán cải tiến mới tốt hơn cho chấm điểm tín dụng và áp dụng cho một trong các tập dữ liệu chấm điểm tín dụng sau:

*   Tập dữ liệu Kaggle: https://www.kaggle.com/c/GiveMeSomeCredit





In [ ]:
print("hello")